# Hands-On: Schätzen und Testen
Hier soll eine beispielhafte Analyse die Themen des Maximum-Likelihood-Schätzens und des Likelihood-Ratio-Testens zusammenfassend veranschaulichen.

Die benutzten Methoden werden größtenteils zur Veranschaulichung direkt im notebook implementiert.
Viele der Methoden hier sind in Statistik-Paketen implementiert und sollten dann eigenen Implementierungen vorgezogen werden.

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from scipy.stats import norm, chi2
from scipy.optimize import minimize

from tqdm.auto import tqdm

In [2]:
%matplotlib widget

In [3]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 100

Zunächst wird ein Beispielexperiment erstellt.

Das Modell besteht aus der Superposition zweier Normal-Verteilungen mit folgenden Parametern:

In [4]:
model_a = norm(0, 1)
model_b = norm(3, 2)

Die PDFs sehen wie folgt aus:

In [5]:
x = np.linspace(-5, 10, 1000)

fig = plt.figure(constrained_layout=True)
plt.plot(x, model_a.pdf(x), label = 'A')
plt.plot(x, model_b.pdf(x), label = 'B')

plt.xlabel(r'$x$')
plt.ylabel(r'Probability Density Function')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Nun soll eine Verteilung `data` aus diesen beiden PDFs erstellt werden, um eine Messung zu simulieren.

Dabei wird $20$ mal aus der Verteilung $A$ und $250$ mal aus der Verteilung $B$ gezogen. $A$ stellt das Signal und $B$ den Untergrund dar.

In [6]:
N_A = 20
N_B = 250

rng = np.random.default_rng(1337)

sample_a = model_a.rvs(size=N_A, random_state=rng)
sample_b = model_b.rvs(size=N_B, random_state=rng)
data = np.append(sample_a, sample_b)

Dies ergibt die folgende Verteilung:

In [7]:
limits = [data.min(), data.max()]

plt.figure(constrained_layout=True)
plt.hist(data, histtype = 'step', label='Total', range=limits, bins=25)
plt.hist(sample_a, histtype = 'step', label='A', range=limits, bins=25)
plt.hist(sample_b, histtype = 'step', label='B', range=limits, bins=25)
plt.legend()

plt.xlabel(r'$x$')
plt.ylabel(r'Observed Events')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Observed Events')

Das Ziel dieses Beispiels ist, das kleine Signal $A$ im deutlich größeren Untergrund $B$ zu messen.

## Erstellen der Likelihood

Die Likelihood (das Produkt über die einzelnen Wahrscheinlichkeiten) ist hier

$\mathcal{L}(f | \boldsymbol{x}) = \prod\limits_i P(f|x_i)$,

wobei die kombinierte PDF $P(f|x_i)$ die normierte Superposition von $A$ und $B$ ist

$P(f|x) = f\cdot A(x) + (1-f)\cdot B(x)$

In [8]:
def pdf(f):
    return f * model_a.pdf(data) + (1 - f) * model_b.pdf(data)


def likelihood(f):
    return np.prod(pdf(f), axis=-1)

Diese Likelihood ist eine Funktion von $f$, dem Anteil (*fraction*) der Daten, welcher die Signalstärke von $A$ gegenüber $B$ beschreibt.

Das Maximum der Funktion sollte dabei bei $f_\mathrm{max} = \frac{N_A}{N_A + N_B}$ liegen.

In [9]:
f_true = N_A / (N_A + N_B)
print(f"{f_true:.4f}")

0.0741


In [10]:
f_space = np.linspace(0, 1, 1000, endpoint=False)

fig = plt.figure(constrained_layout=True)

lh = likelihood(f_space[:, np.newaxis])
f_max = f_space[np.argmax(lh)]

plt.plot(f_space, lh, label=r'$\mathcal{L}(f \,| \mathbf{x})$')

plt.axvline(f_true, ls=':', color='C1', label='$f_\mathrm{true}$')
plt.axvline(f_max, ls='--', color='C2', label='$f_\mathrm{max}$')
plt.legend()

plt.xlabel(r'$f$')
plt.ylabel(r'$\mathcal{L}$')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Der Wertebereich der Likelihood umfasst mehrere Größenordnungen, was die handhabung erschwert.

Außerdem stellen die meisten Numerik-Bibliotheken *Minimierer* und nicht *Maximierer* zur Verfügung.

Wie üblich, verwenden wir daher die **negative Log-Likelihood**.

\begin{equation}
  -\log\mathcal{L}(f|\boldsymbol{x}) = -\sum\limits_i \log\left(fA(x_i) + (1-f)B(x_i)\right)
\end{equation}

In [11]:
def negative_log_likelihood(f):
    return -np.sum(np.log(pdf(f)), axis=-1)

Da der Logarithmus eine monoton steigende Funktion ist, stimmt das Maximum der *log-likelihood* mit dem Maximum der *Likelihood* überein.

Dies kann kurz per Auge überprüft werden.

In [12]:
f_space = np.linspace(0, 1, 1000, endpoint=False)

fig = plt.figure(constrained_layout=True)

nll = negative_log_likelihood(f_space[:, np.newaxis])
f_min = f_space[np.argmin(nll)]

plt.plot(f_space, nll, label=r'$-\log\mathcal{L}(f \,| \mathbf{x})$')

plt.axvline(f_true, ls=':', color='C1', label='$f_\mathrm{true}$')
plt.axvline(f_min, ls='--', color='C2', label='$f_\mathrm{min}$')
plt.legend()

plt.xlabel(r'$f$')
plt.ylabel(r'$-\log\mathcal{L}(f \,| \mathbf{x})$')
plt.show()

print(f_min)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.053


Bei der Suche nach dem Maximum, wird die _ungebinnte_ Likelihood häufig ausgewertet, da stets über jedes Ereignis iteriert wird.

Diese Methode ist leider sehr langsam, vor allem, wenn große Datensätze betrachtet werden.

Daher wird die Likelihood in eine gebinnte Verteilung umformuliert.

In [13]:
def create_linear_bin_edges_and_mids(low, high, n_bins):
    bin_edges = np.linspace(low, high, n_bins + 1)
    bin_mids = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    return bin_edges, bin_mids

bin_edges, bin_mids = create_linear_bin_edges_and_mids(-5, 10, 100)
binned_data, _ = np.histogram(data, bins=bin_edges)

Wir nutzen die kumulative Verteilung (CDF) um die PDF zu diskretisieren.

In [14]:
binned_a = np.diff(model_a.cdf(bin_edges)) / np.diff(bin_edges)
binned_b = np.diff(model_b.cdf(bin_edges)) / np.diff(bin_edges)

Sollte keine analytische CDF bekannt sein sondern nur die PDF und eine effiziente Möglichkeit diese zu samplen, kann auch zufällig gesamplet werden.

In [15]:
binned_a_sampled, _ = np.histogram(model_a.rvs(size=1_000_000, random_state=rng), bins=bin_edges, density=True)
binned_b_sampled, _ = np.histogram(model_b.rvs(size=1_000_000, random_state=rng), bins=bin_edges, density=True)

Hier der Vergleich der beiden Verteilungen zwischen der gebinnten und der ungebinnten Version.

In [16]:
plt.figure(constrained_layout=True)
plt.plot(x, model_a.pdf(x), label = 'A')
plt.stairs(binned_a, bin_edges, label='A binned', zorder=5)

plt.plot(x, model_b.pdf(x), label='B')
plt.stairs(binned_b, bin_edges, label='B binned', zorder=5)

plt.xlabel(r'$x$')
plt.ylabel(r'Probability')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Die gebinnte Likelihood ist ähnlich wie die Ungebinnte definiert, außer dass mehrere Ereignisse, die im selben Bin landen, herausfaktorisiert werden.

Somit muss nur über die Bins und nicht über jedes einzelne Ereignis iteriert werden.

$\mathrm{LLH} = \log\mathcal{L}(f|\vec{x}) = \sum\limits_\mathrm{bins} N_\mathrm{bin} \log\left(fA_\mathrm{bin} + (1-f)B_\mathrm{bin}\right)$

In [17]:
def binned_negative_log_likelihood(f, binned_sample=binned_data):
    mu = f * binned_a + (1 - f) * binned_b
    return -np.sum(binned_sample * np.log(mu), axis=-1)

Wie zu erwarten, gleicht die gebinnte Likelihood der Ungebinnten.

In [18]:
fig = plt.figure()
plt.plot(f_space, negative_log_likelihood(f_space[:, np.newaxis]), label='unbinned')
plt.plot(f_space, binned_negative_log_likelihood(f_space[:, np.newaxis]), ls=':', label='binned', color='C3')

plt.axvline(f_true, color='C1')

plt.xlabel(r'$f$')
plt.ylabel(r'$-\log\mathcal{L}$')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mit einem kurzen Test kann überprüft werden, ob die gebinnte Log-Likelihood die Ungebinnte hinreichend gut beschreibt, sodass ein Minimizer das Maximum findet.

Dabei ist der Performanceunterschied deutlich zu sehen.

In [19]:
minimize_kwargs = {
    "x0": [0.5],
    "bounds": [(0, 1)]
}

In [20]:
%%timeit
result = minimize(negative_log_likelihood, **minimize_kwargs)

5.8 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
result = minimize(negative_log_likelihood, **minimize_kwargs)
result

      fun: 565.1343391188238
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00011369])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 14
      nit: 6
     njev: 7
   status: 0
  success: True
        x: array([0.05268395])

In [22]:
%%timeit
result = minimize(binned_negative_log_likelihood, **minimize_kwargs)

1.41 ms ± 73.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
result = minimize(binned_negative_log_likelihood, **minimize_kwargs)
result

      fun: 565.371073282461
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00014779])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 14
      nit: 6
     njev: 7
   status: 0
  success: True
        x: array([0.05225993])

## Test der Nullhypothese

Da es nun möglich ist den maximal wahrscheinlichen Parameter $f_\mathrm{best}$ zu finden, können nun im nächsten Schritt Hypothesen getestet werden.

Annahme: Wir sind nicht sicher, ob es eine Komponente $A$ in `data` gibt.

Dies führt zur Nullhypothese, dass $f = 0$ der wahre Wert ist, die wir nun mit einem Likelihood-Ratio-Test verwerfen wollen.

Daraus lässt sich die folgende Test-Statistik erstellen:

$\mathcal{TS} = 2 \log \left( \frac{\mathcal{L}(f = f_\mathrm{best})}{\mathcal{L}(f = 0)}\right) = 2\left(\mathrm{LLH}(f = f_\mathrm{best}) - \mathrm{LLH}(f = 0)\right)$

In [24]:
def calculate_ts(sample, f_0=0):
    binned_sample, _ = np.histogram(sample, bins=bin_edges)

    fit_result = minimize(
        binned_negative_log_likelihood,
        args=(binned_sample, ),
        **minimize_kwargs
    )
    
    f_fit = fit_result.x[0]
    nllh = fit_result.fun
    
    return -2 * (nllh - binned_negative_log_likelihood(f_0, binned_sample))

Zum Generieren einer Test-Statistik-Verteilung für die Nullhypothese wird eine große Anzahl an zufällig gezogenen Ereignissen aus $B$ erstellt und für diese der Test-Statistik-Wert berechnet.

In [25]:
n_scrambles = 30000
null_ts = np.empty(n_scrambles)

for i in tqdm(range(n_scrambles)):
    sample = model_b.rvs(size=len(data), random_state=rng)
    null_ts[i] = calculate_ts(sample)

  0%|          | 0/30000 [00:00<?, ?it/s]

Die normierte Nuller-Test-Statistik-Verteilung kann nun mit dem Test-Statistik-Wert der gemessenen Verteilung `data` verglichen werden. Daraus folgt die Signifikanz, mit der die Nullhypothese, dass der Beitrag von $A=0$ ist, verworfen werden kann.

Zum Vergleich ist zudem die $\chi^2$-Verteilung dargestellt. 

Zur Erinnerung, das Wilks-Theorem sagt:

Falls
1. sich die Nullhypothese durch eine lineare Parameter-Transformation als ein 
    Spezialfall der Alternativ-Hypothese darstellen lässt
2. die Anzahl der Beobachtungen gegen unendlich geht
3. Keiner der Parameter einen Extremwert annimmt

ist die Teststatistik $\chi^2$ verteilt. 

In unserem Fall ist $f=0$ ein Extremwert, somit gilt Wilks-Theorem hier nicht, auch nicht für einen wesentlich größeren Datensatz, der der gleichen Verteilung folgt.

Der Likelihood-Ratio-Test ist trotzdem valide, die Abschätzung der P-Values muss aber durch sampling der Test-Statistik für die Nullhypothese bestimmt werden.

In [26]:
observed_ts = calculate_ts(data)

plt.figure()
ts_bin_edges = np.linspace(0, 15, 100)


plt.hist(
    null_ts,
    bins=ts_bin_edges,
    density=True,
    cumulative=-1,
    histtype='step',
    label=r'$\mathcal{TS}$ distribution',
)


plt.axvline(observed_ts, color='C2', ls='--', label = rf'Observed $\mathcal{{TS}} = {observed_ts:.2f}$')

ts_space = np.linspace(0, 15, 1000)

# surfival function = 1 - cdf
plt.plot(ts_space, chi2.sf(ts_space, 1), label = r'$\chi^2$ with $N_\mathrm{DoF} = 1$')
plt.legend()


plt.xlabel(r'$\mathcal{TS}$')
plt.ylabel('p-value')

plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Der p-value bzw. die Signifikanz der Messung gegeben der Hypothese, dass der Beitrag von $A$ zu "data" $0$ ist, kann nun aus der Position des gemessenen Test-Statistik-Wertes in der nuller Test-Statistik abgelesen werden.

In [27]:
def sigma_from_pval(p_value):
    return norm.ppf(1 - p_value / 2)

p_value = np.count_nonzero(null_ts > observed_ts) / len(null_ts)
p_value_chi2 = chi2.sf(observed_ts, 1) 

print(f"p-value {p_value:2.4f} and significance {sigma_from_pval(p_value):2.1f} σ of 'data'")
print(f"p-value {p_value_chi2:2.4f} and significance {sigma_from_pval(p_value_chi2):2.1f} σ according to chi-squared")

p-value 0.0356 and significance 2.1 σ of 'data'
p-value 0.0709 and significance 1.8 σ according to chi-squared


## Berechnung der Konfidenzintervalle

Nun ist es wichtig, neben dem BestFit-Wert auch die entsprechenden Unsicherheitsbereiche mit anzugeben. Wir wissen ja bereits, dass selbst die $2\sigma$ Fehlergrenze nicht mit Null kompatibel ist.

Zunächst wird hierfür die Neyman-Konstruktion erstellt.

Dazu wird der Signalstärkeparameter $f$ kleinschrittig abgefahren und für jeden Wert eine Test-Statistik-Verteilung erstellt/gewürfelt und histogrammiert.

In [28]:
n_ts_bins = 100
ts_bin_edges, ts_bin_mids = create_linear_bin_edges_and_mids(0, 120, n_ts_bins)

n_f_bins = 51
f_bin_edges, f_bin_mids = create_linear_bin_edges_and_mids(0, 0.3, n_f_bins)

neyman_construction = np.empty((n_f_bins, n_ts_bins))

def sample_from_combined_pdf(f, n_samples=len(data)):
    n_a = int(round(f * n_samples))
    n_b = n_samples - n_a
    return np.append(
        model_a.rvs(size=n_a, random_state=rng),
        model_b.rvs(size=n_b, random_state=rng)
    )

def generate_binned_ts_dist(f, n_scrambles=5000):
    ts = np.empty(n_scrambles)
    for idx in range(n_scrambles):
        sample = sample_from_combined_pdf(f)    
        ts[idx] = calculate_ts(sample)
        
    ts_binned, _ = np.histogram(ts, bins=ts_bin_edges, density=True)
    return ts_binned

In [29]:
for idx in tqdm(range(n_f_bins)):
    neyman_construction[idx] = generate_binned_ts_dist(f_bin_mids[idx])

  0%|          | 0/51 [00:00<?, ?it/s]

Das engmaschige Gitter der Neyman-Konstruktion kann gut dargestellt werden.

Gut zu erkennen ist der entstehende Gürtel, da sich für ein größeres Signal die Teststatistikverteilung zu höheren Werten verschiebt. 

In [30]:
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.pcolormesh(ts_bin_edges, f_bin_edges, neyman_construction, norm=LogNorm())

ax.axvline(x=observed_ts, color='C1', ls = '--')
ax.set_xlabel(r"$\mathcal{TS}$")
ax.set_ylabel(r"$f$")
fig.colorbar(im)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Aus dem Gitter wird nun die normierte Spalte betrachtet, indem der gemessene Test-Statistik-Wert liegt.

In [31]:
def get_f_pdf(ts_value):
    idx = np.digitize(ts_value, bins=ts_bin_edges)
    f_dist = neyman_construction[:, idx]
    f_dist /= np.sum(f_dist)
    return f_dist

f_pdf = get_f_pdf(observed_ts)

plt.figure(constrained_layout=True)

plt.stairs(f_pdf, f_bin_edges, label=r"PDF$(f | \mathcal{TS}_\mathrm{meas.})$")

plt.axvline(f_true, ls = '--', color='C1', label=r"$f_\mathrm{true}$")
plt.axvline(result.x, ls = '--', color='C2', label=r"$f_\mathrm{best}$")
plt.xlabel(r"$f$")
plt.ylabel(r"Probability")

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mit einem gewählten Konfidenzlevel $\alpha=0.9$ können die Grenzen eines Zentralintervalls über die Quantile der Verteilung des Signalstärkeparameters $f$ bestimmt werden.

Dabei wird solange das höchste benachbarte bin miteingeschlossen, bis das gewünschte Konfidenzlevel erreicht ist.

In [32]:
def get_upper_and_lower_idx(dist, alpha):
 
    cumulated = np.cumsum(dist)  
    outside = (1 - alpha) / 2
    
    # lower bound 
    indices, = np.where(cumulated >= outside)
    if len(indices) > 0:
        f_idx_low = indices[0]
    else:
        f_idx_low = 0
       
    # upper bound
    indices, = np.where(cumulated >= 1 - outside)
    if len(indices) > 0:
        f_idx_up = indices[0]
    else:
        f_idx_up = len(indices) - 1
    
    return f_idx_low, f_idx_up


def get_central_interval(alpha, bin_edges=f_bin_edges):
    low_idx, up_idx = get_upper_and_lower_idx(f_pdf, alpha)
    bin_mids = 0.5 * (f_bin_edges[:-1] + f_bin_edges[1:])
    lim_low = bin_mids[low_idx]
    lim_up = bin_mids[up_idx]
    
    return lim_low, lim_up

f_low, f_up = get_central_interval(0.9)
print(f"lower bound: {f_low:.4f}")
print(f"upper bound: {f_up:.4f}")

lower bound: 0.0206
upper bound: 0.1088


Somit ergibt sich das folgende Ergebnis:


In [33]:
print("In den Messdaten wurde mit einem p-Value von {:.3f} und somit einer Signifikanz von {:.1f} sigma ausgeschlossen, dass sie nur aus Untergrundereignissen bestehen.".format(p_value, sigma_from_pval(p_value)))
print("Der Signalstärkeparameter wurde dabei zu {:.3f} bestimmt mit dem 90 Prozent Konfidenzintervall ({:.4f}, {:.4f})".format(result.x[0], f_low, f_up))

In den Messdaten wurde mit einem p-Value von 0.036 und somit einer Signifikanz von 2.1 sigma ausgeschlossen, dass sie nur aus Untergrundereignissen bestehen.
Der Signalstärkeparameter wurde dabei zu 0.052 bestimmt mit dem 90 Prozent Konfidenzintervall (0.0206, 0.1088)


In [34]:
plt.figure(constrained_layout=True)

plt.stairs(f_pdf, f_bin_edges, label=r"PDF$(f | \mathcal{TS}_\mathrm{meas.})$")

plt.axvline(f_true, ls='--', color='C1', label=r"$f_\mathrm{true}$")
plt.axvline(result.x, ls='--', color='C2', label=r"$f_\mathrm{best}$")

plt.axvline(f_low, ls='--', color="k", label=r"$90\,\%$ Confidence")
plt.axvline(f_up, ls='--', color="k")

plt.xlabel(r"$f$")
plt.ylabel(r"Probability")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Inverser Hypothesentest

Die bisherige Berechnung ist eine konservative Vorgehensweise, da in der Test-Statistik gegen die Nullhypothese verglichen wird.

Nun soll die Test-Statistik jeweils gegen den injezierten Messwert verglichen werden. Hier wird also ein anderer, alternativer Hypothesentest durchgeführt.

$\mathcal{TS}_{f_0} = 2 \log \left( \frac{\mathcal{L}(f = f_\mathrm{best})}{\mathcal{L}(f = f_0)}\right) = 2\left(\mathrm{LLH}(f = f_\mathrm{best}) - \mathrm{LLH}(f = f_0)\right)$

Zunächst wird in einem feinen Grid wieder $f$ abgefahren und die Teststatistik erstellt/gewürfelt.

In [35]:
n_f_bins_inv = 100
f_bin_edges_inv, f_bin_mids_inv = create_linear_bin_edges_and_mids(0, 0.25, n_f_bins_inv)
n_scrambles_inv = 1000


def generate_ts_dist(f_inject, n_scrambles=n_scrambles_inv):
    ts = np.empty(n_scrambles)
  
    for ts_idx in range(n_scrambles):
        sample = sample_from_combined_pdf(f_inject)
        ts[ts_idx] = calculate_ts(sample, f_inject)

    return ts

neyman_construction_inv = np.empty((n_f_bins_inv, n_scrambles_inv))
for f_idx in tqdm(range(n_f_bins_inv)):
    neyman_construction_inv[f_idx] = generate_ts_dist(f_bin_edges_inv[f_idx])

  0%|          | 0/100 [00:00<?, ?it/s]

Anschließend wird das Konfidenzlevel $\alpha$ festgelegt und die kritischen Werte $\zeta$ berechnet. $\zeta$ bezeichnet dabei die Grenze des Quantils der Teststatistik, bei dem der Wert des Konfidenzlevels erreicht wird. Dieser Wert wird für jede vom injezierten $f$ abhängige Teststatistikverteilung berechnet.

$\int\limits_0^\zeta -2\log\left( \frac{\mathcal{L}(\mu_t)}{\mathcal{L}(\mu_b)} \right) \mathrm{d}TS = \alpha$

In [36]:
alpha = 90
critical_values = np.percentile(neyman_construction_inv, alpha, axis=1)

Nach dem Bestimmen der kritischen Werte, können nun die TS-Verteilungen histogrammiert werden, um das äquivalent zur Neyman Konstruktion zu erstellen.

In [37]:
n_ts_bins_inv = 100
ts_bin_edges_inv, ts_bin_mids_inv = create_linear_bin_edges_and_mids(0, 20, n_ts_bins_inv)
neyman_construction_binned = np.empty((n_f_bins_inv, n_ts_bins_inv))

for idx in range(n_f_bins_inv):
    neyman_construction_binned[idx], _ = np.histogram(
        neyman_construction_inv[idx],
        bins=ts_bin_edges_inv,
        density=True,
    )

Diese kleinschrittig in $f$ abgefahrenen Teststatistikverteilungen können gut mit den kritischen Werten dargestellt werden. Diese können des Weiteren mit dem Wilks Theorem verglichen werden.

Gut zu erkennen ist, wie für höhere Signalbeiträge, also einer höheren Statistik des Signals in den Messdaten, die kritischen Werte gegen das Wilks Theorem konvergieren.

In [38]:
fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(1, 1, 1)

im = ax.pcolormesh(ts_bin_edges_inv, f_bin_edges_inv, neyman_construction_binned, norm=LogNorm())

ax.plot(critical_values, f_bin_mids_inv, color = 'r', ls='--', label='critical values')
ax.axvline(x = chi2.ppf(0.9, 1) / 2, ls='--', color="k", label='Wilks Theorem')

ax.set_xlabel(r"$\mathcal{TS}$")
ax.set_ylabel(r"$f$")
ax.legend()

fig.colorbar(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Schließlich kann mithilfe eines Likelihood-Scans aus den Schnittpunkten der gemessenen Teststatistikverteilung und den kritischen Werten das $90\%$ Konfidenz Intervall bestimmt werden.

In [39]:
llh_scan = [calculate_ts(data, f_0=f_bin_mids_inv[idx]) for idx in range(n_f_bins_inv)]

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(1, 1, 1)

ax.plot(f_bin_mids_inv, critical_values, color = 'r', ls = '--', label='critical values')
ax.plot(f_bin_mids_inv, llh_scan, label='llh scan')


ax.axvline(x = f_true, ls = '--', color='C1', label=r"$f_\mathrm{true}$")
ax.axvline(x = result.x, ls = '--', color='C2', label=r"$f_\mathrm{best}$")

ax.set_xlabel(r"$f$") 
ax.set_ylabel(r"$TS$")
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ein Vergleich mit den vorher berechneten Grenzen:

In [40]:
f_min = f_bin_mids_inv[llh_scan < critical_values].min()
f_max = f_bin_mids_inv[llh_scan < critical_values].max()

print("f_low_old: {:.3f}, f_up_old: {:.3f}".format(f_low, f_up))
print("f_low_new: {:.3f}, f_up_new:  {:.3f}".format(f_min, f_max))

f_low_old: 0.021, f_up_old: 0.109
f_low_new: 0.016, f_up_new:  0.096
